# Introduction to `albert`

This notebook is intended to introduce the core functionality of `albert`.

In [1]:
from albert.tensor import Tensor
from albert.scalar import Scalar
from albert.algebra import Add, Mul
from albert.base import Base
from albert.index import Index
from albert.expression import Expression
from albert.symmetry import Permutation, Symmetry
from albert.misc import from_string

## Basics

The primary building blocks of `albert` are computational graphs which can be classified as [arborescences](https://en.wikipedia.org/wiki/Arborescence_%28graph_theory%29); that is, a directed acyclic graph (DAG) in which every vertex has a single parent and shares the same root node. This structure is sufficient to define any sum of tensor contractions obeying the [Einstein summation convention](https://en.wikipedia.org/wiki/Einstein_notation) using just four types of vertices:

1. a scalar node, consisting of a single scalar factor;
2. a tensor node, consisting of a tensor and the associated indices;
3. an addition node, consisting of a collection of nodes that are added;
4. a multiplication node, consisting of a collection of nodes that are contracted, according to their indices.

These nodes (vertices) are implemented in `albert` as `Scalar`, `Tensor`, `Add`, and `Mul`, respectively. The first two of these nodes represent leaves in the graph, as they do not have child nodes themselves. The two algebraic nodes are internal nodes, each with children that define the arguments to the respective operand. Each class is a subclass of a `Base` object which offers a powerful toolbox for walking the graph and constructing, analysing, and applying tensor contractions.

The simplest node is the `Scalar`, which just requires the value:

In [2]:
s = Scalar(2.0)
s

2

The `Tensor` object requires specification of the indices, and the name (label) of the tensor.

In [3]:
indices = (Index("i"), Index("j"))
x = Tensor(*indices, name="x")
x

x(i,j)

The `Add` object requires specification of the tensors that are being added together.

In [4]:
add = Add(
    Tensor(*indices, name="x"),
    Tensor(*indices, name="y"),
)
add

x(i,j) + y(i,j)

The arguments must have the same indices (within transposition).

In [5]:
error = None
try:
    add = Add(
        Tensor(Index("i"), Index("j"), name="x"),
        Tensor(Index("i"), Index("k"), name="y"),
    )
except ValueError as e:
    error = e
error

ValueError('External indices in additions must be equal.')

The `Mul` requires specification of the tensors that are being contracted.

In [6]:
mul = Mul(
    Tensor(Index("i"), Index("j"), name="x"),
    Tensor(Index("j"), Index("k"), name="y"),
)
mul

x(i,j) * y(j,k)

The indices must obey the Einstein summation convention.

In [7]:
error = None
try:
    mul = Mul(
        Tensor(Index("i"), Index("j"), name="x"),
        Tensor(Index("i"), Index("k"), name="y"),
        Tensor(Index("i"), Index("l"), name="z"),
    )
except ValueError as e:
    error = e
error

ValueError('Input arguments are not a valid Einstein notation.  Each index must appear at most twice.')

Note that this error can be silenced by setting `albert.ALLOW_NON_EINSTEIN_NOTATION` to `True`. This is experimental and *will* break things, but in simple cases of static expression construction, allows at least basic support for expressions that do not obey Einstein summation notation.

Graphs can be constructed by either

- instantiating the node classes as above, along with the children as arguments;
- using the `Base.factory` method of each node, with the same signature as the class constructor;
- using the overloaded operators `*` and `+`.

The latter two methods perform basic simplifications in trivial cases, for example, collecting like terms or detecting necessary zeros.

In [8]:
x = Tensor(Index("i"), Index("j"), name="x")
Mul.factory(Scalar(0.0), x)

0

In [9]:
x = Tensor(Index("i"), Index("j"), name="x")
y = Tensor(Index("j"), Index("k"), name="y")
z = Tensor(Index("i"), Index("k"), name="z")
rhs = (x * y) + z
rhs

(x(i,j) * y(j,k)) + z(i,k)

The default `repr` of the graphs displays their algebraic form. For large nested expressions this can become unintuitive, and instead there is a function to view the graph in a tree-like representation.

In [10]:
x = Tensor(Index("i"), Index("j"), name="x")
y = Tensor(Index("j"), Index("k"), name="y")
z = Tensor(Index("i"), Index("k"), name="z")
c = Tensor(Index("a"), Index("k"), name="c")
d = Tensor(Index("i"), Index("a"), name="d")
rhs = (x * y) + z
rhs = (rhs * d) + (rhs * d) + c
print(rhs.tree_repr())

Add
├── Mul
│   ├── Add
│   │   ├── Mul
│   │   │   ├── x(i,j)
│   │   │   └── y(j,k)
│   │   └── z(i,k)
│   └── d(i,a)
├── Mul
│   ├── Add
│   │   ├── Mul
│   │   │   ├── x(i,j)
│   │   │   └── y(j,k)
│   │   └── z(i,k)
│   └── d(i,a)
└── c(a,k)


## Simplification

Graphs have a series of simple methods for basic simplification

- `expand`, to expand brackets;
- `squeeze`, to remove any redundant algebraic operations;
- `collect`, to collect like terms and sum them via modified scalar factors.

Expansion via `expand` converts arbitrarily nested graphs into the form `Add[Mul[Tensor | Scalar]]` by expanding brackets.

In [11]:
rhs = rhs.expand()
print(rhs.tree_repr())

Add
├── Mul
│   ├── x(i,j)
│   ├── y(j,k)
│   └── d(i,a)
├── Mul
│   ├── z(i,k)
│   └── d(i,a)
├── Mul
│   ├── x(i,j)
│   ├── y(j,k)
│   └── d(i,a)
├── Mul
│   ├── z(i,k)
│   └── d(i,a)
└── Mul
    └── c(a,k)


Squeezing via `squeeze` will remove any redundant nodes; for example, the final `Mul` in this example has a single argument, and this node can be replaced with the tensor argument itself.

In [12]:
rhs = rhs.squeeze()
print(rhs.tree_repr())

Add
├── Mul
│   ├── x(i,j)
│   ├── y(j,k)
│   └── d(i,a)
├── Mul
│   ├── z(i,k)
│   └── d(i,a)
├── Mul
│   ├── x(i,j)
│   ├── y(j,k)
│   └── d(i,a)
├── Mul
│   ├── z(i,k)
│   └── d(i,a)
└── c(a,k)


Collection via `collect` will collect like terms and insert a multiplication by a scalar factor to account for their frequency. This may not be sufficient to reverse `expand`, as it is not a true factorisation.

In [13]:
rhs = rhs.collect()
print(rhs.tree_repr())

Add
├── Mul
│   ├── 2
│   ├── x(i,j)
│   ├── y(j,k)
│   └── d(i,a)
├── Mul
│   ├── 2
│   ├── z(i,k)
│   └── d(i,a)
└── c(a,k)


## Permutation

Nodes in graphs have external indices; those being the indices that are not contracted over within that particular node. For a `Tensor` those are simply the specified indices, for an `Add` the equal indices of each argument, and for a `Mul` the external indices within the Einstein summation convention.

The external indices of a node can be permuted using the `permute_indices` method.

In [14]:
tensor = Tensor(Index("i"), Index("j"), Index("k"), name="x")
tensor.permute_indices((0, 2, 1))

x(i,k,j)

More generally, all indices (both internal and external) can be permuted according to an explicit mapping using the `map_indices` method.

In [15]:
rhs = rhs.map_indices(
    {
        Index("i"): Index("j"),
        Index("j"): Index("i"),
        Index("a"): Index("w"),
    }
)
print(rhs.tree_repr())

Add
├── Mul
│   ├── 2
│   ├── x(j,i)
│   ├── y(i,k)
│   └── d(j,w)
├── Mul
│   ├── 2
│   ├── z(j,k)
│   └── d(j,w)
└── c(w,k)


Nodes can also be permuted using a `Permutation` object, which contains the permutation and sign.

In [16]:
tensor = Tensor(Index("i"), Index("j"), name="x")
permutation = Permutation((1, 0), +1)
permutation(tensor)

x(j,i)

A collection of `Permutation` objects can be composed into a `Symmetry` object to represent the permutational symmetry of a tensor. These symmetry groups can be passed during the initialisation of the tensor, and then the resulting graph canonicalised under those symmetries using the `canonicalise` method.

In [17]:
symmetry = Symmetry(
    Permutation((0, 1), +1),
    Permutation((1, 0), +1),
)
rhs = Tensor(Index("i"), Index("j"), name="x", symmetry=symmetry)
rhs += Tensor(Index("j"), Index("i"), name="x", symmetry=symmetry)
rhs.canonicalise().collect()

2 * x(i,j)

Setting `albert.INFER_ALGEBRA_SYMMETRIES` to `True` will allow `albert` to infer the symmetries of algebraic nodes (`Add` or `Mul`) on construction, from the symmetries provided for the children nodes. This is often very useful for determining the permutational symmetries of output tensors or intermediates, when the symmetry of the inputs is known, but it may impact performance noticeably and is therefore disabled by default.

## Traversal

Since the graphs are essentially n-ary generalisations of a binary tree, we can use familiar tree traversal methods to perform depth-first search over the graph to find specific nodes, or do surgery on the graph itself, using the four functions

- `search`, to search for nodes matching some type filter;
- `find`, to search for the first node matching some type filter;
- `delete`, to set the value of a node matching some type filter to zero;
- `apply`, to call a function on a node matching some type filter, and replace the node with the result.

The type filters can be a type to match the instance of, a tuple of types, or a function that evaluates to `True` or `False` when called with the node as the argument. Since nodes are immutable objects, each method returns a copy of the original node, which may contain references to existing nodes.

Each function also has a `depth` and `order` argument that can be used to control the maximum depth of the search, and whether the search proceeds in post- or pre-order.

In [18]:
rhs = from_string("(a(i,l) * b(k,l) + c(i,k)) * d(j,k)")
print(rhs.tree_repr())

Mul
├── Add
│   ├── Mul
│   │   ├── a(i,l)
│   │   └── b(k,l)
│   └── c(i,k)
└── d(j,k)


In [19]:
list(rhs.search(Tensor))

[a(i,l), b(k,l), c(i,k), d(j,k)]

In [20]:
rhs.find(lambda node: isinstance(node, Tensor) and node.name == "c")

c(i,k)

In [21]:
rhs.delete(lambda node: isinstance(node, Tensor) and node.name == "c")

a(i,l) * b(k,l) * d(j,k)

In [22]:
rhs.apply(lambda tensor: tensor.copy(name=tensor.name.upper()), Tensor)

((A(i,l) * B(k,l)) + C(i,k)) * D(j,k)

The `apply` function should be used for the majority of cases where one wishes to algebraically manipulate the expression. Many complex substitutions, morphisms, and other manipulations are possible through this approach. Another example is symbolically representing tensor factorisations.

In [24]:
rhs = from_string("a(i,k) * a(k,j)")

count = 0

def factorise_a(tensor: Tensor) -> Tensor:
    global count
    if tensor.name == "a":
        i, j = tensor.indices
        x = Index(f"x{count}")
        count += 1
        return Tensor(i, x, name="u") * Tensor(j, x, name="v")
    return tensor

rhs.apply(factorise_a, Tensor)

u(i,x0) * v(k,x0) * u(k,x1) * v(j,x1)

Lastly, the `children` property can also be used to directly access the children of a node, allowing the user to implement custom traversals for more complex requirements.

## Evaluation

Graph expressions can be numerically evaluated without the need to use the code generation functionality, using the `evaluate` function. This requires the arrays corresponding to each tensor name to be passed in a dictionary, and an `einsum` driver function must be provided. In more advanced cases where the `Index` objects possess spaces, the dictionary of arrays must be a nested dictionary where each entry is itself a dictionary mapping spaces to arrays.

In [ ]:
import numpy as np

rhs = from_string("(a(i,l) * b(k,l) + c(i,k)) * d(j,k)")

np.random.seed(123)
i, j, k, l = 6, 7, 8, 9
a = np.random.random((i, l))
b = np.random.random((k, l))
c = np.random.random((i, k))
d = np.random.random((j, k))

reference = (a @ b.T + c) @ d.T
result = rhs.evaluate(dict(a=a, b=b, c=c, d=d), np.einsum)
np.allclose(reference, result)

Alternatively, the `code` module provides template code generators that are intended to be subclassed to fit specific code generation requirements. Using one of the default code generators, we can generate a list of `np.einsum` calls. The `__call__` method of the code generator takes the desired function name, a list of tensors to be returned from the function, and a list of `Expression` objects, which each wrap a LHS tensor and a RHS definition.

In [ ]:
from albert.code.einsum import EinsumCodeGenerator
import sys

lhs = from_string("x(i,j)")
rhs = from_string("(a(i,l) * b(k,l) + c(i,k)) * d(j,k)")
expr = Expression(lhs, rhs)

codegen = EinsumCodeGenerator(stdout=sys.stdout)
codegen.preamble()
codegen("my_function", [lhs], [expr])
codegen.postamble()

## Serialisation

Essentially everything in `albert` inherits from the `Serialisable` base class, which means that they are serialisable, deserialisable, hashable, comparable, and sortable.

There is a global intern table carrying a weak reference from the hash of a node to its value, and constructing nodes via their `factory` methods will make a cache request. This means that instantiation in this way can return nodes identical to existing ones, permitting efficiency and memory optimisations.

In [ ]:
x1 = Tensor(Index("i"), Index("j"), name="x")
x2 = Tensor(Index("i"), Index("j"), name="x")
(x1 == x2, x1 is x2)

In [ ]:
x1 = Tensor.factory(Index("i"), Index("j"), name="x")
x2 = Tensor.factory(Index("i"), Index("j"), name="x")
(x1 == x2, x1 is x2)

Comparison and hashing are dispatched via the `_hashable_fields` method, which yields identical fields for any subclass of `Base`. This means that any pair of nodes can be compared or sorted, and equality takes advantage of short-circuited evaluation.

In [ ]:
nodes = [
    from_string("a(i,j) * c(j,k)"),
    from_string("a(i,j)"),
    from_string("1"),
    from_string("a(i,j) + b(i,j)"),
]
sorted(nodes)

Using an internal JSON data format, `as_json` and `from_json` can be used for serialisation and deserialisation. This allows arbitrary tensor expressions to be saved and reused in the native `albert` format, rather than only according to code generated formats. 

In [ ]:
rhs = from_string("a(i,j) * b(j,k)")
rhs.as_json()

In [ ]:
loaded = Base.from_json(rhs.as_json())
loaded

In [ ]:
loaded == rhs